In [1]:
import pandas as pd
df = pd.read_csv("Cars.csv")
classification = df['clazz'].unique()

In [2]:
from sklearn.model_selection import train_test_split
train , test = train_test_split(df , test_size=0.2)

In [3]:
BaseClassProbability , num = {} , {}
classification = df['clazz'].unique()
for classes in classification : 
    BaseClassProbability[classes] = 0
for col in train['clazz'] :
    BaseClassProbability[col] += 1
for classes in classification : 
    num[classes] = BaseClassProbability[classes]
    BaseClassProbability[classes]/=len(train)
print(BaseClassProbability)

{'unacc': 0.7018813314037626, 'acc': 0.22214182344428365, 'vgood': 0.03762662807525326, 'good': 0.03835021707670044}


In [4]:
keys =  df.keys()
cols = []
for k in keys :
    if k != 'clazz': cols.append(k)
print(cols)

['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']


In [5]:
attribute_probability = {}
for name in cols :
    attribute_probability[name] = {}
    for classes in classification :
        attribute_probability[name][classes] = {}
    features = train[name].unique()
    for i in features : 
        for classes in classification : attribute_probability[name][classes][i] = 0
    for i in train.index :
        attribute_probability[name][train['clazz'][i]][train[name][i]] += 1
    for i in features : 
        for classes in classification : 
            attribute_probability[name][classes][i] = attribute_probability[name][classes][i]/num[classes]

In [9]:
from pprint import pprint
pprint(attribute_probability)

{'buying': {'acc': {'high': 0.2899022801302932,
                    'low': 0.23778501628664495,
                    'med': 0.28338762214983715,
                    'vhigh': 0.18892508143322476},
            'good': {'high': 0.0,
                     'low': 0.660377358490566,
                     'med': 0.33962264150943394,
                     'vhigh': 0.0},
            'unacc': {'high': 0.2670103092783505,
                      'low': 0.21443298969072164,
                      'med': 0.22061855670103092,
                      'vhigh': 0.2979381443298969},
            'vgood': {'high': 0.0,
                      'low': 0.5961538461538461,
                      'med': 0.40384615384615385,
                      'vhigh': 0.0}},
 'doors': {'acc': {'2': 0.20195439739413681,
                   '3': 0.2671009771986971,
                   '4': 0.25732899022801303,
                   '5more': 0.2736156351791531},
           'good': {'2': 0.22641509433962265,
                    '3': 0.245283018

In [7]:
def BayesPredict(param):
    maxscore , prediction = 0 , ""
    for classes in classification :
        score = BaseClassProbability[classes]
        for name in cols : 
            score *= attribute_probability[name][classes][param[name]]
        if score > maxscore : 
            maxscore = score
            prediction = classes
    return prediction

In [8]:
acc = 0
for i in range(len(test)) :
    row = test.iloc[i]
    prediction = BayesPredict(row)
    if row['clazz'] == prediction : 
        acc += 1
print("Accuracy of Naive Bayes  : " + str(acc/len(test)))

Accuracy of Naive Bayes  : 0.8583815028901735
